In [1]:
import os
import string
import tempfile
import tensorflow as tf
import numpy as np
import pandas as pd

from tensorflow.python.keras.preprocessing import sequence
from tensorboard import summary as summary_lib
from sklearn.model_selection import train_test_split


tf.logging.set_verbosity(tf.logging.INFO)
print(tf.__version__)

1.13.1


In [2]:
from pymystem3 import Mystem
from tqdm import tqdm_notebook,tqdm_pandas,tqdm
import re
from sklearn.feature_extraction.text import CountVectorizer

from gensim.models import KeyedVectors

import nltk
import json
from pathlib import Path

import tf_metrics
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.text import text_to_word_sequence

from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences


from multiprocessing import cpu_count, Pool
import gc

In [3]:
tqdm_pandas(tqdm)

## Loading file     

In [4]:
df_train = pd.read_csv('pikabu.csv',sep=',')

In [5]:
df_train.head(20)

,context,answer
0,Убрал 600 литров мусора в реликтовом лесу. Ты...,Спасибо.
1,Убрал 600 литров мусора в реликтовом лесу. Ты ...,Приедь к нам в Мурманск пожалуйста.
2,Убрал 600 литров мусора в реликтовом лесу. Ты ...,Тебе платят за это?
3,Ты просто большой молодец. Спасибо. Тебе платя...,"За посты на Пикабу? Да, платят. Достаточно при..."
4,Спасибо. Тебе платят за это? За посты на Пикаб...,Не нажимается сук
5,Ты просто большой молодец. Спасибо. Тебе платя...,Лично Путин
6,Спасибо. Тебе платят за это? Лично Путин,дарт вейдар
7,Убрал 600 литров мусора в реликтовом лесу. Ты...,"ну всё, теперь можно не убирать за собой, Чист..."
8,Убрал 600 литров мусора в реликтовом лесу. Ты ...,Зачем заминусили человека? Очевидная ирония же...
9,Убрал 600 литров мусора в реликтовом лесу.,"Герой, про которого я бы читал комиксы от dc и..."


In [14]:
df_train.count()

context    21486165
answer     21486162
dtype: int64

In [22]:
len(df_train.loc[df_train['context'].apply(len)>410,'context'][271713].split())

67

## Preprocessing

{4: 'убирать_VERB'}

In [6]:
from nltk.corpus import stopwords
sw_ru = set(stopwords.words('russian'))

In [7]:
mystem = Mystem()

In [11]:
context = df_train['context'].values

In [43]:
def index_tokenizer(tokenizer, corpus):
    #voc_set = set()
    res = np.fromiter(map(tokenizer,tqdm_notebook(corpus[:20]),),dtype=str)
    voc_set = np.unique(res)
    print(voc_set)

In [42]:
def fu(x):
    return np.apply_along_axis(text_prep_tags,1,np.reshape(x,(-1,1)))
        
def parallelize(data,partitions):

    data_split = np.array_split(data, partitions)
    pool = Pool(partitions)
    #data = np.stack(pool.apply(np.apply_along_axis,text_prep_tags,1,np.reshape(data_split,(-1,1))))
    data = np.concatenate(pool.map(fu,data_split))
    pool.close()
    pool.join()
    return data

ValueError: not enough values to unpack (expected 3, got 2)

In [56]:
%time np_context = parallelize(context[:20000],4)

CPU times: user 173 ms, sys: 342 ms, total: 515 ms
Wall time: 13.2 s


In [48]:
text_id_vocab = {}
id_text_vocab = {}
prev_word = None

def text_prep_tags(text):
   
    #print(text)
    text = text[0]
    upos_map = {'A':'ADJ','ADV':'ADV','ADVPRO':'ADV','ANUM':'ADJ','APRO':'DET','COM':'ADJ','CONJ':'SCON','INTJ':'INTJ','NONLEX':'X','NUM':'NUM','PART':'PART','PR':'ADP','S':'NOUN','SPRO':'PRON','UNKN':'X' ,'V':'VERB'}
    text = text.lower()
    result = np.array([])
    
    
    # Убираем лишние символы
    #text = re.sub(r'[;,]',r' ',text).strip()
    text = re.sub(r'[^\w\s\.]',r'',text).strip()   
    #text = [token.text for token in razdel.tokenize(text)]
    # Делаем лемматизацию       
#     text = [lemma for lemma in mystem.lemmatize(text) if not lemma.isspace() and lemma not in sw_ru
#             and lemma.strip() not in ['.','..','...']]
    
#     if id_text_vocab != {}:
#         prev_word = id_text_vocab[max(id_text_vocab.keys())]
#     else:       
#         prev_word = None
        
    for item in mystem.analyze(text):
      #  print(item)
        token = None
        if item.get('analysis'):
            lemma = item['analysis'][0]['lex']
            pos = re.split('[=,]', item['analysis'][0]['gr'])[0]
            #and lemma not in sw_ru
            if not lemma.isspace()  and lemma.strip() not in ['.','..','...'] and lemma not in sw_ru:
     
                token = f'{lemma}_{upos_map[pos]}'
        else:
            lem_text = item["text"]
            if not lem_text.isspace() and lem_text.strip() not in ['.','..','...'] and lem_text not in sw_ru:
            
                token = f'{lem_text}_UNKN'
            
        if token:    
            #result.append(token)
            result = np.append(result,token)
            
#             if prev_word:
#                 text_id_vocab[token] = text_id_vocab[prev_word] + 1
#                 id_text_vocab[text_id_vocab[prev_word] + 1] = token
#             else:
#                 text_vocab[token] = 4
#                 id_text_vocab[4] = token
#             prev_word = token

    
    
    
    # Лемматизатор неправильно разбивает слова с дефисом, поэтому исправляем это
#     if '-' in text:
#         for l in range(len(text)):
#             if text[l] == '-':
#                 text[l] = f'{text[l-1]}-{text[l+1]}'
#                 text[l-1] = text[l+1] = text[l]
    zer = np.empty([40], dtype="U25")
    zer[:] = '<PAD>'
    size = result.shape[0]
    if size>40:
        zer = result[:40]
    else:
        zer[:size] = result
        
    return zer

In [89]:
class Vocab:
    def __init__(self):
        
        self.char2idx = {'<PAD>':0,'<START>':1,'<END>':2,'<UNK>':3}
        self.idx2char = {i:ch for ch,i in self.char2idx.items()}
    
    def save(self, path='.'):
        json_ch_idx = json.dumps(self.char2idx)
        with open(os.path.join(path,"char2idx.json"),"w") as f:
            f.write(json_ch_idx)
        json_idx_ch = json.dumps(self.idx2char)
        with open(os.path.join(path,"idx2char.json"),"w") as f:
            f.write(json_idx_ch)
    
    def load(self, path='.'):       
        with open(os.path.join(path,"char2idx.json"),"r") as f:
            self.char2idx = json.loads(f.read())        
        with open(os.path.join(path,"idx2char.json"),"r") as f:
            self.idx2char = json.loads(f.read())
        
    
    def indx_tokenize_from_files(self, huge_parts):
        if not os.path.exists('indexed_token_files'):
            os.mkdir('indexed_token_files')
        for i in tqdm_notebook(range(huge_parts)):
            with np.load(f'tokenized_files/np_tokens{i}.npz') as data:
                np_idxs = np.apply_along_axis(np.vectorize(self.char2idx.get),1,data['arr'])
                print(np_idxs)
                np.savez_compressed(f'indexed_token_files/indx_tokens{i}',arr=np_idxs)
                
        
    
    def indx_detokenize(self, sequence):
        return ''.join([self.idx2char[idx] for idx in sequence])
    
    def tokenize(self, data, tokenizer, huge_parts, save_to_file=True):
        huge_splits = np.array_split(data, huge_parts)
        partitions = cpu_count()
        for split_number,huge_split in enumerate(tqdm_notebook(huge_splits)):
            print(huge_split.shape)          
            data_split = np.array_split(huge_split, partitions)
            pool = Pool(partitions)

            res_data = np.concatenate(pool.map(tokenizer,data_split))
            pool.close()
            pool.join()
            
            print(res_data.shape)
            self._extend_vocab(res_data)
            
            print(res_data)
            
            if save_to_file:
                np.savez_compressed(f'tokenized_files/np_tokens{split_number}',arr=res_data)             
                del res_data
                gc.collect()
                
                
    def _extend_vocab(self,token_matrix):
        start_idx = len(self.char2idx)
        for idx,token in np.ndenumerate(np.unique(token_matrix)):
            if token not in self.char2idx:
                self.char2idx[token] = start_idx + idx[0]
        
        self.idx2char = {i:ch for ch,i in self.char2idx.items()}
                
        
    
    def __len__(self):
        return len(self.char2idx)

In [14]:
def fu(x):
    return np.apply_along_axis(text_prep_tags,1,np.reshape(x,(-1,1)))
voc = Vocab()
%time voc.tokenize(context[:4000000],fu,4)

(1000000,)
(1000000, 40)
[['убирать_VERB' '600_UNKN' 'литр_NOUN' ... '' '' '']
 ['убирать_VERB' '600_UNKN' 'литр_NOUN' ... '' '' '']
 ['убирать_VERB' '600_UNKN' 'литр_NOUN' ... '' '' '']
 ...
 ['гейб_NOUN' 'молодец_NOUN' 'давно_ADV' ... '' '' '']
 ['гейб_NOUN' 'молодец_NOUN' 'давно_ADV' ... '' '' '']
 ['гейб_NOUN' 'молодец_NOUN' 'давно_ADV' ... '' '' '']]
(1000000,)
(1000000, 40)
[['молодец_NOUN' 'самый_DET' 'начало_NOUN' ... '' '' '']
 ['это_PRON' 'человек_NOUN' 'виноватый_ADJ' ... '' '' '']
 ['беда_NOUN' 'гринлайт_NOUN' 'плюсонуть_VERB' ... '' '' '']
 ...
 ['сложно_ADV' 'олдфаг_NOUN' 'зарегистрировать_VERB' ... '' '' '']
 ['зарегистрировать_VERB' 'месяц_NOUN' 'олдфаг_NOUN' ... '' '' '']
 ['это_PRON' 'читать_VERB' 'сидеть_VERB' ... '' '' '']]
(1000000,)
(1000000, 40)
[['сидеть_VERB' 'тег_NOUN' 'жесть_NOUN' ... '' '' '']
 ['сложно_ADV' 'олдфаг_NOUN' 'зарегистрировать_VERB' ... '' '' '']
 ['сложно_ADV' 'олдфаг_NOUN' 'зарегистрировать_VERB' ... '' '' '']
 ...
 ['пн_UNKN' '2_UNKN' 'геома_

In [90]:
voc1=Vocab()
voc1.load()


In [76]:
def yy(x):
    print(x)
    return np.vectorize(voc1.char2idx.get)(x)
np.apply_along_axis(yy,1,np.array([['молодец_NOUN', 'самый_DET' ,'начало_NOUN'],
 ['это_PRON' ,'человек_NOUN', 'виноватый_ADJ'],
 ['беда_NOUN', 'гринлайт_NOUN' ,'плюсонуть_VERB']]))

['молодец_NOUN' 'самый_DET' 'начало_NOUN']
['это_PRON' 'человек_NOUN' 'виноватый_ADJ']
['беда_NOUN' 'гринлайт_NOUN' 'плюсонуть_VERB']


array([[ 99766, 149023, 105369],
       [188210, 181013,  45919],
       [ 36253,  56266, 126309]])

In [88]:
voc1.char2idx

{'<PAD>': 0,
 '<START>': 1,
 '<END>': 2,
 '<UNK>': 3,
 '': 4,
 '    _ _UNKN': 5,
 '    __   _UNKN': 6,
 '   _ _UNKN': 7,
 '   __\n_UNKN': 8,
 '   __ __UNKN': 9,
 '  _\n_UNKN': 10,
 '  _  _UNKN': 11,
 '  _ _\n_UNKN': 12,
 '  _ _UNKN': 13,
 '  __\n_UNKN': 14,
 '  __  _UNKN': 15,
 '  __  __UNKN': 16,
 '  __ _UNKN': 17,
 '  __UNKN': 18,
 '  ___\n_UNKN': 19,
 '  ___UNKN': 20,
 '  ヽ\n_UNKN': 21,
 '  ヽ _UNKN': 22,
 '  ヽ ー__UNKN': 23,
 '  ヽ_UNKN': 24,
 ' _\n_UNKN': 25,
 ' _   _UNKN': 26,
 ' _  _UNKN': 27,
 ' _ _ _\n_UNKN': 28,
 ' _ _ _ _ _ _\n_UNKN': 29,
 ' _ _ _ _ _ _ _ _ _ _ _ _ ': 30,
 ' _ _ _ _ _ _ _UNKN': 31,
 ' _ _ _ _UNKN': 32,
 ' _ _ _UNKN': 33,
 ' _ _UNKN': 34,
 ' _ __UNKN': 35,
 ' _ ヽ _UNKN': 36,
 ' __\n_UNKN': 37,
 ' __  _UNKN': 38,
 ' __ _ _\n_UNKN': 39,
 ' __ _UNKN': 40,
 ' __ __UNKN': 41,
 ' __UNKN': 42,
 ' ___\n_UNKN': 43,
 ' ___ _UNKN': 44,
 ' ___UNKN': 45,
 ' ____\n_UNKN': 46,
 ' ____ _UNKN': 47,
 ' ____UNKN': 48,
 ' _____\n_UNKN': 49,
 ' _____ _ _UNKN': 50,
 ' _____ _UNKN': 5

In [91]:
voc1.indx_tokenize_from_files(4)

[[169730   6934  91754 ...      4      4      4]
 [169730   6934  91754 ...      4      4      4]
 [169730   6934  91754 ...      4      4      4]
 ...
 [ 52516  99766  57344 ...      4      4      4]
 [ 52516  99766  57344 ...      4      4      4]
 [ 52516  99766  57344 ...      4      4      4]]
[[ 99766 149023 105369 ...      4      4      4]
 [188210 181013  45919 ...      4      4      4]
 [ 36253  56266 126309 ...      4      4      4]
 ...
 [154435 114713  71694 ...      4      4      4]
 [ 71694  97254 114713 ...      4      4      4]
 [188210 182252 152074 ...      4      4      4]]
[[152074 164183  67894 ...      4      4      4]
 [154435 114713  71694 ...      4      4      4]
 [154435 114713  71694 ...      4      4      4]
 ...
 [126392   4256 328320 ...      4      4      4]
 [ 37280 153558 149710 ...      4      4      4]
 [138822 353514 119067 ...      4      4      4]]
[[110355 172004 407414 ...      4      4      4]
 [110355 172004 407414 ...      4      4      4]
 [

str

424590

In [47]:
np_arr = None
for i in range(1):
    with np.load(f'tokenized_files/np_tokens{i}.npz') as data:
        if np_arr is not None:
            np_arr = np.append(np_arr,data['arr'],axis=0)
        else:
            np_arr = data['arr']


In [81]:
np_arr = None
for i in range(1):
    with np.load(f'indexed_token_files/indx_tokens{i}.npz') as data:
        if np_arr is not None:
            np_arr = np.append(np_arr,data['arr'],axis=0)
        else:
            np_arr = data['arr']

In [82]:
np_arr.shape

(1000000, 40)

In [83]:
np_arr.nbytes/1000000

320.0

array([[128897, 155086, 150350, ...,      4,      4,      4],
       [111158,   2771,  42423, ...,  11618, 115962,  42423],
       [158623, 111158,  73738, ...,      4,      4,      4],
       ...,
       [160165, 132439,  89121, ...,      4,      4,      4],
       [ 89121, 188210, 123192, ...,      4,      4,      4],
       [ 54942, 170538, 172776, ...,      4,      4,      4]])

In [63]:
%time np.unique(np_context)[100

CPU times: user 264 ms, sys: 10.4 ms, total: 274 ms
Wall time: 260 ms


'139_UNKN'

## TF model

In [ ]:
word2vec = KeyedVectors.load_word2vec_format('../../model.bin', binary=True)

In [ ]:
model_dir = './model'
vocab_size = len(voc)
sentence_size = 40
embedding_size = 300

In [9]:
def input_fn(x,  params, is_training):
    dataset = tf.data.Dataset.from_tensor_slices(x)

    if is_training:
        dataset = dataset.shuffle(buffer_size=params['buffer_size'])
        dataset = dataset.repeat(count=params['num_epochs'])

    dataset = dataset.batch(params['batch_size'])
    dataset = dataset.prefetch(buffer_size=2)
    iterator = dataset.make_one_shot_iterator()
    return iterator.get_next()
    #return dataset

In [ ]:

def initializer(shape=None, dtype=None, partition_info=None):    
    vocab_dict = count_v.vocabulary_
    embedding_matrix = np.random.uniform(-1, 1, size=(vocab_size+1, embedding_size))
    num_loaded = 0
    for w, i in vocab_dict.items():
        v = None
        try:
            v = word2vec[w]
        except KeyError: # не нашли такой токен в словаре
                pass
        if v is not None :
            embedding_matrix[i+1] = v
            num_loaded += 1
   
    embedding_matrix = embedding_matrix.astype(np.float32)
    embedding_matrix[0] = np.zeros(300)
    return embedding_matrix

In [ ]:
def model_fn(features, labels, mode, params):  
    
    # Compute predictions.
    net = params['net']
    
    encoded_features = {}  
    
    with tf.variable_scope('encoder'):
        encoded_features['anchor'] = net(features['anchor'])
    with tf.variable_scope('encoder', reuse=True):
        encoded_features['positive'] = net(features['positive'])
    with tf.variable_scope('encoder', reuse=True):
        encoded_features['negative'] = net(features['negative'])
    
    
    #predicted_classes = tf.argmax(logits, 1)
    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {
            'class_ids': predicted_classes[:, tf.newaxis],
            'probabilities': tf.nn.softmax(logits),
            'logits': logits,
        }
        return tf.estimator.EstimatorSpec(mode, predictions=predictions)
    
    # Compute loss.
    loss = my_triplet_loss(encoded_features,params['margin'])
    
#     # Compute evaluation metrics.
#     accuracy = tf.metrics.accuracy(labels=labels,
#                                predictions=predicted_classes,
#                                name='acc_op')
#     f1 = tf_metrics.f1(labels=labels,
#                                predictions=predicted_classes,num_classes=3,average='micro')
    
#     metrics = {'accuracy': accuracy,'f1':f1}
#     tf.summary.scalar('accuracy', accuracy[1])
    
#     # Compute evaluation
#     if mode == tf.estimator.ModeKeys.EVAL:
#         return tf.estimator.EstimatorSpec(
#             mode, loss=loss, eval_metric_ops=metrics)
    
    optimizer = tf.train.AdagradOptimizer(learning_rate=0.1)
    
    train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
    
    return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)